<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

HOME#10

In [249]:
import sqlite3
import pandas as pd

In [250]:
con = sqlite3.connect(':memory:')

In [251]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [253]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [254]:
tasks = 0

**Все задания нужно выполнить только с помощью sql**

**_1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес._**

In [255]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName AS Название,
    Country AS Страна,
    Address AS Адрес 
    FROM Customers
    WHERE Country='Germany' OR Country='France' OR City='Madrid'
    ''',
    con,
)

,Название,Страна,Адрес
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


**_2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей._**

In [256]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country AS Страна, 
    COUNT(Country) AS Количество_записей 
    FROM Customers
    GROUP BY Country
    ORDER BY Количество_записей DESC
    LIMIT 3
    ''',
    con, 
)

,Страна,Количество_записей
0,USA,13
1,Germany,11
2,France,11


**_3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления._**

In [257]:
tasks+=1
pd.read_sql(
    '''
     SELECT Название_перевозчика,
            MAX(Дата) AS Дата
     FROM(
          SELECT ShipperName AS Название_перевозчика, 
                OrderDate AS Дата
          FROM Shippers 
               JOIN Orders USING(ShipperID)
          ORDER BY OrderID
          LIMIT 10
          )
    ''', 
    con, 
)


,Название_перевозчика,Дата
0,Federal Shipping,16.07.1996


**_4. Выберите самый дорогой заказ, выведите список товаров с их ценами._**

In [258]:
tasks+=1
pd.read_sql(
    '''     
        WITH table_1 AS 
        (SELECT MAX(Sum_Price) AS Max_Sum,
                OrderID
        FROM (
              SELECT OrderID, 
              SUM(Price) AS Sum_Price
              FROM OrderDetails
                   JOIN Products USING(ProductID)
              GROUP BY OrderID
              ORDER BY SUM(Price) DESC
              )
         )
         
        SELECT OrderDetails.OrderID AS N_заказа,
               ProductName AS Список_товаров,
               Price AS Цена
        FROM table_1 
             JOIN OrderDetails USING(OrderID)
             JOIN Products ON OrderDetails.ProductID=Products.ProductID
      
                                         
    ''',
    con,
)


,N_заказа,Список_товаров,Цена
0,10360,Rössle Sauerkraut,45.60
1,10360,Thüringer Rostbratwurst,123.79
2,10360,Côte de Blaye,263.50
3,10360,Maxilaku,20.00
4,10360,Tourtière,7.45


**_5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов._**

In [259]:
tasks+=1
pd.read_sql(
    '''
       WITH table_1 AS
         (SELECT ProductID,MAX(Количество) AS Max,Название_товара
          FROM(
             SELECT ProductName AS Название_товара, 
             SUM(Quantity) AS Количество,
             Products.ProductID
             FROM OrderDetails
                  JOIN Products USING(ProductID)
             GROUP BY ProductID  
             ORDER BY Количество DESC
             
             )
          )
           SELECT Название_товара,OrderID AS N_заказа,Quantity AS Количество_единиц
           FROM table_1 
               JOIN OrderDetails USING(ProductID)
               JOIN Products ON OrderDetails.ProductID=Products.ProductID
           ORDER BY Количество_единиц DESC
       
    ''',
    con,
)
        

,Название_товара,N_заказа,Количество_единиц
0,Gorgonzola Telino,10359,70
1,Gorgonzola Telino,10390,60
2,Gorgonzola Telino,10342,56
3,Gorgonzola Telino,10376,42
4,Gorgonzola Telino,10272,40
5,Gorgonzola Telino,10393,32
6,Gorgonzola Telino,10356,30
7,Gorgonzola Telino,10374,30
8,Gorgonzola Telino,10335,25
9,Gorgonzola Telino,10253,20


**_6.Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон._**

In [260]:
tasks+=1
pd.read_sql(
     '''
       SELECT 
       SupplierName AS Поставщик,
       Country AS Страна,
       ContactName AS Контактное_лицо,
       Phone AS Телефон, 
       SUM(Quantity) AS Количество_заказов
       
       FROM Products
            JOIN Suppliers USING(SupplierID)
            JOIN OrderDetails USING(ProductID)
            
       GROUP BY SupplierID
       ORDER BY Количество_заказов DESC
       LIMIT 5
      
      ''',
    con,
)

,Поставщик,Страна,Контактное_лицо,Телефон,Количество_заказов
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343,1148
1,Norske Meierier,Norway,Beate Vileid,(0)2-953010,836
2,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323,780
3,Gai pâturage,France,Eliane Noz,38.76.98.06,776
4,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755,776


**_7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму._**

In [261]:
tasks+=1
pd.read_sql(
     '''
       SELECT Страна,Категория,MAX(Сумма) AS Сумма
       FROM(
            SELECT Country as Страна,
            CategoryName AS Категория,
            SUM(Price) AS Сумма
            FROM Products 
               JOIN Categories USING(CategoryID)
               JOIN OrderDetails USING(ProductID)
               JOIN Orders ON OrderDetails.OrderID=Orders.OrderID
               JOIN Customers ON Orders.CustomerID=Customers.CustomerID
            WHERE Country='Brazil'
            GROUP BY CategoryName
            ORDER BY Сумма DESC
            )
      ''',
    con,
)

,Страна,Категория,Сумма
0,Brazil,Beverages,405.75


**_8.Какая разница в стоимости между самым дорогим и самым дешевым заказом из США._**

In [262]:
tasks+=1
pd.read_sql(
     '''
       SELECT MAX(Стоимость_заказа)-MIN(Стоимость_заказа) AS Разница
       FROM(
            SELECT SUM(Price) AS Стоимость_заказа
            FROM Products 
               JOIN Categories USING(CategoryID)
               JOIN OrderDetails USING(ProductID)
               JOIN Orders ON OrderDetails.OrderID=Orders.OrderID
               JOIN Customers ON Orders.CustomerID=Customers.CustomerID
            WHERE Country='USA'
            GROUP BY OrderDetails.OrderID
            ORDER BY SUM(Price) DESC
            )
      ''',
    con,
)

,Разница
0,334.09


**_9.Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке._**

In [263]:
tasks+=1
pd.read_sql(
    '''
             SELECT 
             COUNT(OrderID) AS Количество_заказов,
             FirstName||' '|| LastName AS Имя_Фамилия
            
             FROM Orders
                  JOIN OrderDetails USING(OrderID)
                  JOIN Employees USING(EmployeeID)
             GROUP BY EmployeeID
             ORDER BY substr(BirthDate, length(BirthDate)-4) DESC,substr(BirthDate, length(BirthDate)-7) DESC
             LIMIT 3
             
    ''',
    con, 
)


,Количество_заказов,Имя_Фамилия
0,19,Anne Dodsworth
1,78,Nancy Davolio
2,74,Janet Leverling


**_10.Сколько банок крабового мяса всего было заказано._**

In [266]:
tasks+=1
pd.read_sql(
    '''
             SELECT SUM(Quantity) AS Всего_банок
            
             FROM Products
                  JOIN OrderDetails USING(ProductID)
             WHERE ProductName LIKE '%Crab%'
             
    ''',
    con, 
)


,Всего_банок
0,256


In [265]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
